In [2]:
from langchain_core.documents import Document


In [3]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders import TextLoader 

In [4]:
## loading pdf files to train the model

In [44]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    "../data/pdf",
    glob = "**/*.pdf", 
    loader_cls = PyMuPDFLoader,
    show_progress = False
)

pdf_documents = dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'calibre 2.49.0 [http://calibre-ebook.com]', 'creator': 'calibre (2.49.0) [http://calibre-ebook.com]', 'creationdate': '2016-01-20T18:18:21+00:00', 'source': '..\\data\\pdf\\gestalt-therapy.pdf', 'file_path': '..\\data\\pdf\\gestalt-therapy.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': 'Gestalt Therapy', 'author': 'John O. Stevens', 'subject': '', 'keywords': '', 'moddate': '2016-01-20T12:29:16-06:00', 'trapped': '', 'modDate': "D:20160120122916-06'00'", 'creationDate': "D:20160120181821+00'00'", 'page': 0}, page_content=''),
 Document(metadata={'producer': 'calibre 2.49.0 [http://calibre-ebook.com]', 'creator': 'calibre (2.49.0) [http://calibre-ebook.com]', 'creationdate': '2016-01-20T18:18:21+00:00', 'source': '..\\data\\pdf\\gestalt-therapy.pdf', 'file_path': '..\\data\\pdf\\gestalt-therapy.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': 'Gestalt Therapy', 'author': 'John O. Stevens', 'subject': '', 'keywords': '', 'moddate': '2016-01-

In [11]:
###chunking 

In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def chunk_documents(documents, chunk_size=2500, chunk_overlap=50):
    """
    Split documents into smaller chunks for better RAG performance.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ".", " "]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print("\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs

In [35]:
# ...existing code...
import gc
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader

def is_chunked(docs):
    return bool(docs and isinstance(docs[0].metadata, dict) and docs[0].metadata.get("chunk_id"))

# Inspect current variables
print("Has variable 'chunks' in globals():", 'chunks' in globals())
print("Is 'chunks' chunked?:", is_chunked(globals().get('chunks')) if 'chunks' in globals() else False)
print("Is 'pdf_documents' chunked?:", is_chunked(globals().get('pdf_documents')) if 'pdf_documents' in globals() else False)

# Delete in-memory chunked objects
if 'chunks' in globals():
    del chunks
    print("Deleted variable 'chunks'")

# If pdf_documents was overwritten with chunks, reload originals from PDF folder
pdf_dir = "../data/pdf"
if 'pdf_documents' in globals() and is_chunked(pdf_documents):
    print("pdf_documents looks chunked — reloading original PDFs from", pdf_dir)
    dir_loader = DirectoryLoader(
        pdf_dir,
        glob="**/*.pdf",
        loader_cls=PyMuPDFLoader,
        show_progress=False
    )
    pdf_documents = dir_loader.load()
    print("Reloaded pdf_documents, count =", len(pdf_documents))

# force python to free memory
gc.collect()
print("Cleanup done.")
# ...existing code...

Has variable 'chunks' in globals(): True
Is 'chunks' chunked?: False
Is 'pdf_documents' chunked?: False
Deleted variable 'chunks'
Cleanup done.


In [48]:
chunks=chunk_documents(pdf_documents)
chunks

Split 4906 documents into 5371 chunks

Example chunk:
Content: Gestalt Therapy
John O. Stevens...
Metadata: {'producer': 'calibre 2.49.0 [http://calibre-ebook.com]', 'creator': 'calibre (2.49.0) [http://calibre-ebook.com]', 'creationdate': '2016-01-20T18:18:21+00:00', 'source': '..\\data\\pdf\\gestalt-therapy.pdf', 'file_path': '..\\data\\pdf\\gestalt-therapy.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': 'Gestalt Therapy', 'author': 'John O. Stevens', 'subject': '', 'keywords': '', 'moddate': '2016-01-20T12:29:16-06:00', 'trapped': '', 'modDate': "D:20160120122916-06'00'", 'creationDate': "D:20160120181821+00'00'", 'page': 1}


[Document(metadata={'producer': 'calibre 2.49.0 [http://calibre-ebook.com]', 'creator': 'calibre (2.49.0) [http://calibre-ebook.com]', 'creationdate': '2016-01-20T18:18:21+00:00', 'source': '..\\data\\pdf\\gestalt-therapy.pdf', 'file_path': '..\\data\\pdf\\gestalt-therapy.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': 'Gestalt Therapy', 'author': 'John O. Stevens', 'subject': '', 'keywords': '', 'moddate': '2016-01-20T12:29:16-06:00', 'trapped': '', 'modDate': "D:20160120122916-06'00'", 'creationDate': "D:20160120181821+00'00'", 'page': 1}, page_content='Gestalt Therapy\nJohn O. Stevens'),
 Document(metadata={'producer': 'calibre 2.49.0 [http://calibre-ebook.com]', 'creator': 'calibre (2.49.0) [http://calibre-ebook.com]', 'creationdate': '2016-01-20T18:18:21+00:00', 'source': '..\\data\\pdf\\gestalt-therapy.pdf', 'file_path': '..\\data\\pdf\\gestalt-therapy.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': 'Gestalt Therapy', 'author': 'John O. Stevens', 'subject': '', 'key

In [7]:
### Embedding and vectorstore

In [14]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    def __init__(self,model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: Huggingface model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

## initialize the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384
Model loaded successfully. Embedding dimension: 384


In [16]:
### VectorStore


In [17]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """Initialize the vector store
        
        Args:
            collection_name Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()
    
    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # create persistent Chromadb client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match nuber of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents tpo vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
        

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 4


In [50]:
texts=[doc.page_content for doc in chunks]

embeddings = embedding_manager.generate_embeddings(texts)

vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 5371 texts...


Batches:   0%|          | 0/168 [00:00<?, ?it/s]

Generated embeddings with shape: (5371, 384)
Adding 5371 documents to vector store...
Successfully added 5371 documents tpo vector store
Total documents in collection: 5375
Successfully added 5371 documents tpo vector store
Total documents in collection: 5375


In [52]:
### Retriever pipeline from VectorStore
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
    
    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold

        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'if': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("no documents found")
            
            return retrieved_docs
        
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []
        
rag_retriever=RAGRetriever(vectorstore,embedding_manager)
        


In [53]:
rag_retriever

In [54]:
rag_retriever.retrieve("i feel depressed ")

Retrieving documents for query: 'i feel depressed '
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
Retrieved 0 documents (after filtering)


[]

In [32]:
rag_retriever.retrieve("what is the distance between earth and moon")

Retrieving documents for query: 'what is the distance between earth and moon'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving documents for query: 'what is the distance between earth and moon'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]

In [55]:
from datetime import datetime


def save_user_query(query: str, metadata: dict = None):
    """Embed and save a user query to the vector store for future context-aware retrieval."""
    if metadata is None:
        metadata = {}
    metadata['type'] = 'user_query'
    metadata['timestamp'] = datetime.utcnow().isoformat()

    # Create a LangChain Document for the query
    doc = Document(page_content=query, metadata=metadata)

    # Generate embedding for the query
    embedding = embedding_manager.generate_embeddings([query])

    # Add to vector store
    vectorstore.add_documents([doc], embedding)
    print(f"Saved user query: '{query}'")

# Example usage:
# save_user_query("What is the capital of France?")

In [56]:
def handle_user_query(query: str, top_k: int = 5, score_threshold: float = 0.7):
    """Retrieve context for a user query or save it if no similar context exists."""
    results = rag_retriever.retrieve(query, top_k=top_k, score_threshold=score_threshold)
    if not results:
        print("No similar context found. Saving query to vector store.")
        save_user_query(query)
        return None
    else:
        print(f"Found {len(results)} similar context(s). Returning context.")
        return results

# Example usage:
# context = handle_user_query("What is the capital of France?")

In [82]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = "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"

llm=ChatGroq(groq_api_key=groq_api_key,model_name="qwen3-32b",temperature=0.1,max_tokens=1024)

def rag_simple(query, retriever, llm, top_k=3):
    # retrieve the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        save_user_query(query)
        print("No relevant context found. Answering without context.")

    # generate the answer using GROQ LLM
    prompt = f"""use the following context to answer the question like a therapist.
        Context:
        {context}

        Question: {query}

        Answer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    return response.content

In [83]:
answer=rag_simple("i feel very depressed",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'i feel very depressed'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


NotFoundError: Error code: 404 - {'error': {'message': 'The model `qwen3-32b` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'code': 'model_not_found'}}